## Round 1 
* Updating existing charts made by Tiffany. https://gtfs-digest--cal-itp-data-analyses.netlify.app/
* cd rt_segment_speeds && pip install altair_transform && pip install -r requirements.txt && cd ../_shared_utils && make setup_env

Links
* https://github.com/cal-itp/data-analyses/issues/1059
* https://docs.google.com/document/d/1I1WiqlmU06W6iLCi7cZQrOCLILkrEfABEkcU0Jys7f0/edit
* https://route-speeds--cal-itp-data-analyses.netlify.app/name_bay-area-511-muni-schedule/0__report__name_bay-area-511-muni-schedule
* https://posit-dev.github.io/great-tables/get-started/nanoplots.html
* https://docs.pola.rs/py-polars/html/reference/api/polars.from_pandas.html
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_rt_scheduled_utils.py
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_threshold_utils.py

In [ ]:
import _report_utils
import _section2_utils as section2
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS, SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)
from typing import List, Union

from altair_transform.extract import extract_transform
from altair_transform.transform import visit
from altair_transform.utils import to_dataframe

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [ ]:
import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [ ]:
org_name = "City and County of San Francisco"

In [ ]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

### Original File 

In [ ]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [ ]:
og = pd.read_parquet(schd_vp_url)

In [ ]:
og.columns

### Checkout Duplicates

In [ ]:
df = section2.load_schedule_vp_metrics(org_name)

In [ ]:
len(df)

In [ ]:
df["Route"].unique()

### One Route

In [ ]:
one_route = df.loc[df["Route"] == '43 MASONIC']

In [ ]:
one_route.shape

In [ ]:
one_route_all_day = one_route.loc[one_route["Period"] == "all_day"]

In [ ]:
one_route_all_day["Route"].values[0]

#### Explanation Charts

In [ ]:
def divider_chart(df: pd.DataFrame, text):
    df = df.head(1)
    # Create a text chart using Altair
    chart = (
        alt.Chart(df)
        .mark_text(
            align="center",
            baseline="middle",
            fontSize=12,
            text=text,
        )
        .properties(width=500, height=100)
    )

    return chart

In [ ]:
divider_chart(df, "The charts below describe the quality of riding this route.")

#### Cleaning up Code

In [ ]:
def set_y_axis(df, y_col):
    if "%" in y_col:
        max_y = 100

    elif "VP" in y_col:
        max_y = 3
    elif "Minute" in y_col:
        max_y = round(df[y_col].max())
    else:
        max_y = round(df[y_col].max(), -1) + 5
    return max_y

#### Avg Scheduled Minutes (Done)
* How come the average_scheduled_minutes is a lot shorter than total_rt_service_minutes and total_scheduled_service_minutes?
* Kind of a boring chart? Should compare actual run time compared to service minutes?

In [ ]:
def grouped_bar_chart(
    df: pd.DataFrame,
    color_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        color_col,
        y_col,
    ]

    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart
    else:
        df = section2.clean_data_charts(df, y_col)
        chart = (
            alt.Chart(df)
            .mark_bar(size=10)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=["Grouped by Direction ID", "Date"],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(f"{y_col}:Q", title=_report_utils.labeling(y_col)),
                xOffset=alt.X(
                    f"{offset_col}:N", title=_report_utils.labeling(offset_col)
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(
                        range=_report_utils.red_green_yellow,
                    ),
                ),
                tooltip=tooltip_cols,
            )
        )
        chart = (chart).properties(
            title={
                "text": [title],
                "subtitle": [subtitle],
            },
            width=500,
            height=300,
        )

        return chart

In [ ]:
"""grouped_bar_chart(
    df=one_route_all_day,
    color_col="Direction",
    y_col="Average Scheduled Service (trip minutes)",
    offset_col="Direction",
    title=readable_dict["avg_scheduled_min_graph"]["title"],
    subtitle=readable_dict["avg_scheduled_min_graph"]["subtitle"],
)"""

#### Timeliness #2 (Done)

In [ ]:
def timeliness_trips(df: pd.DataFrame):
    to_keep = [
        "Date",
        "Organization",
        "Direction",
        "Period",
        "Route",
        "# Early Arrival Trips",
        "# On-Time Trips",
        "# Late Trips",
        "# Trips with VP",
    ]
    df = df.loc[df["Period"] != "All Day"]
    df2 = df[to_keep]

    melted_df = df2.melt(
        id_vars=[
            "Date",
            "Organization",
            "Route",
            "Period",
            "Direction",
        ],
        value_vars=[
            "# Early Arrival Trips",
            "# On-Time Trips",
            "# Late Trips",
        ],
    )
    return melted_df

In [ ]:
timeliness_test = timeliness_trips(one_route)

In [ ]:
def base_facet_chart(
    df: pd.DataFrame,
    y_col: str,
    color_col: str,
    facet_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        y_col,
        color_col,
    ]

    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart
    else:
        max_y = set_y_axis(df, y_col)
        df = section2.clean_data_charts(df, y_col)
        chart = (
            (
                alt.Chart(df)
                .mark_bar(size=15, clip=True)
                .encode(
                    x=alt.X(
                        "yearmonthdate(Date):O",
                        title=["Date"],
                        axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                    ),
                    y=alt.Y(
                        f"{y_col}:Q",
                        title=_report_utils.labeling(y_col),
                        scale=alt.Scale(domain=[0, max_y]),
                    ),
                    color=alt.Color(
                        f"{color_col}:N",
                        title=_report_utils.labeling(color_col),
                        scale=alt.Scale(range=_report_utils.red_green_yellow),
                    ),
                    tooltip=tooltip_cols,
                )
            )
            .facet(
                column=alt.Column(
                    f"{facet_col}:N",
                )
            )
            .properties(
                title={
                    "text": title,
                    "subtitle": subtitle,
                }
            )
        )
        return chart

In [ ]:
"""base_facet_chart(
    timeliness_test.loc[timeliness_test["Direction"] == 1],
    "value",
    "variable",
    "Period",
    readable_dict["timeliness_trips_dir_1_graph"]["title"],
    readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
)"""

#### Frequency (Done)
* Maybe shouldn't be a chart since there doesn't seem to be a lot of data for this across a lot of the routes?
* What does frequency mean?
* Simplify down to not take direction_id into consideration?

In [ ]:
one_route.head(1)

In [ ]:
def frequency_chart(df: pd.DataFrame):
    if len(df) == 0:
        text_chart = create_data_unavailable_chart()
        return text_chart

    else:
        df["Frequency in Minutes"] = (
            "A trip going this direction comes every "
            + df.frequency_in_minutes.astype(int).astype(str)
            + " minutes"
        )
        chart = (
            alt.Chart(df, width=180, height=alt.Step(10))
            .mark_bar()
            .encode(
                alt.Y(
                    "yearmonthdate(Date):O",
                    title="Date",
                    axis=alt.Axis(format="%b %Y"),
                ),
                alt.X(
                    "frequency_in_minutes:Q",
                    title=_report_utils.labeling("frequency_in_minutes"),
                    axis=None,
                ),
                alt.Color(
                    "frequency_in_minutes:Q",
                    scale=alt.Scale(range=_report_utils.green_red_yellow),
                ).title(_report_utils.labeling("frequency_in_minutes")),
                alt.Row("Period:N")
                .title(_report_utils.labeling("Period"))
                .header(labelAngle=0),
                alt.Column("Direction:N").title(_report_utils.labeling("Direction")),
                tooltip=["Date", "Frequency in Minutes", "Period", "Direction"],
            )
        )
        chart = chart.properties(
            title={
                "text": readable_dict["frequency_graph"]["title"],
                "subtitle": readable_dict["frequency_graph"]["subtitle"],
            }
        )
        return chart

In [ ]:
# frequency_chart(one_route)

#### Speed MPH (Done)
* Needs a different type of chart.

In [ ]:
def base_facet_line(
    df: pd.DataFrame, y_col: str, title: str, subtitle: str
) -> alt.Chart:
    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart
    else:
        max_y = set_y_axis(df, y_col)

        df = section2.clean_data_charts(df, y_col)
        tooltip_cols = [
            "Route",
            "Direction",
            "Period",
            f"{y_col}_str",
        ]

        chart = (
            alt.Chart(df)
            .mark_line(size=5)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title="Date",
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    "Period:N",
                    title=_report_utils.labeling("Period"),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=tooltip_cols,
            )
        )

        chart = chart.properties(width=250, height=300)
        chart = chart.facet(
            column=alt.Column("Direction:N", title=_report_utils.labeling("Direction")),
        ).properties(
            title={
                "text": [title],
                "subtitle": [subtitle],
            }
        )
        return chart

In [ ]:
"""base_facet_line(
    one_route,
    "Speed (MPH)",
    readable_dict["speed_graph"]["title"],
    readable_dict["speed_graph"]["subtitle"],
)"""

#### VP per Minute 

In [ ]:
def base_facet_with_ruler_chart(
    df: pd.DataFrame,
    y_col: str,
    ruler_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        y_col,
    ]

    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart
    else:
        max_y = set_y_axis(df, y_col)
        ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"mean({ruler_col}):Q")
        )
        chart = (
            alt.Chart(df)
            .mark_bar(size=15, clip=True)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=["Date"],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=df[tooltip_cols].columns.tolist(),
            )
        )

        chart = chart + ruler
        chart = chart.facet(column=alt.Column("Direction:N",)).properties(
            title={
                "text": title,
                "subtitle": [subtitle],
            }
        )

        return chart

In [ ]:
"""base_facet_with_ruler_chart(
    one_route_all_day,
    "Average VP per Minute",
    "ruler_for_vp_per_min",
    readable_dict["vp_per_min_graph"]["title"],
    readable_dict["vp_per_min_graph"]["subtitle"],
)"""

#### Spatial Accuracy (Done)
* Multiple it by 100? Should this be rounded or not?

In [ ]:
"""base_facet_with_ruler_chart(
    one_route_all_day,
    "% VP within Scheduled Shape",
    "ruler_100_pct",
    readable_dict["spatial_accuracy_graph"]["title"],
    readable_dict["spatial_accuracy_graph"]["title"],
)"""

#### % RT journey with 1+/2+ vp (goal line = 100%) - use all_day, one chart shared for 1+ and 2+ (Done need subtitle)

In [ ]:
def pct_vp_journey(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    to_keep = [
        "Date",
        "Organization",
        "Direction",
        col1,
        col2,
        "Route",
        "Period",
        "ruler_100_pct",
    ]
    df2 = df[to_keep]

    df3 = df2.melt(
        id_vars=[
            "Date",
            "Organization",
            "Route",
            "Direction",
            "Period",
            "ruler_100_pct",
        ],
        value_vars=[col1, col2],
    )

    df3 = df3.rename(
        columns={"variable": "Category", "value": "% of Actual Trip Minutes"}
    )
    return df3

In [ ]:
rt_pct = pct_vp_journey(
    one_route_all_day,
    "% Actual Trip Minutes with 1+ VP per Minute",
    "% Actual Trip Minutes with 2+ VP per Minute",
)

In [ ]:
rt_pct.columns

In [ ]:
def base_facet_circle(
    df: pd.DataFrame,
    y_col: str,
    color_col: str,
    ruler_col: str,
    title: str,
    subtitle: str,
) -> alt.Chart:

    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Date",
        f"{y_col}_str",
        color_col,
    ]

    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart
    else:
        max_y = set_y_axis(df, y_col)
        df = section2.clean_data_charts(df, y_col)
        ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"ruler_100_pct:Q")
        )

        chart = (
            alt.Chart(df)
            .mark_circle(size=150)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title="Date",
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=tooltip_cols,
            )
        )

        chart = chart + ruler
        chart = chart.facet(
            column=alt.Column("Direction:N", title=_report_utils.labeling("Direction")),
        ).properties(
            title={
                "text": [title],
                "subtitle": [subtitle],
            }
        )
        return chart

In [ ]:
"""base_facet_circle(
    rt_pct,
    "% of Actual Trip Minutes",
    "Category",
    "ruler_100_pct",
    readable_dict["rt_vp_per_min_graph"]["title"],
    readable_dict["rt_vp_per_min_graph"]["subtitle"],
)"""

#### Text

In [ ]:
def route_stats(df: pd.DataFrame) -> pd.DataFrame:
    most_recent_date = df["Date"].max()
    route_merge_cols = ["Route", "Direction"]

    all_day_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "All Day")][
        route_merge_cols
        + [
            "Average Scheduled Service (trip minutes)",
            "Average Stop Distance (miles)",
            "# scheduled trips",
            "GTFS Availability",
        ]
    ]

    peak_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "Peak")][
        route_merge_cols + ["Speed (MPH)", "# scheduled trips", "Trips per Hour"]
    ].rename(
        columns={
            "Speed (MPH)": "peak_avg_speed",
            "# scheduled trips": "peak_scheduled_trips",
            "Trips per Hour": "peak_hourly_freq",
        }
    )

    offpeak_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "Offpeak")][
        route_merge_cols + ["Speed (MPH)", "# scheduled trips", "Trips per Hour"]
    ].rename(
        columns={
            "Speed (MPH)": "offpeak_avg_speed",
            "# scheduled trips": "offpeak_scheduled_trips",
            "frequency": "offpeak_hourly_freq",
        }
    )

    table_df = (
        pd.merge(all_day_stats, peak_stats, on=route_merge_cols, how="outer")
        .merge(offpeak_stats, on=route_merge_cols, how="outer")
        .sort_values(["Route", "Direction"])
        .reset_index(drop=True)
    )

    numeric_cols = table_df.select_dtypes(include="number").columns
    table_df[numeric_cols] = table_df[numeric_cols].fillna(0)
    table_df.columns = table_df.columns.str.title().str.replace("_", " ")
    return table_df

In [ ]:
"""table_df = route_stats(one_route)"""

In [ ]:
def create_text_table(df: pd.DataFrame, direction: float):

    df = df.loc[df["Direction"] == direction].drop_duplicates().reset_index(drop=True)

    if len(df) == 0:
        text_chart = section2.create_data_unavailable_chart()
        return text_chart

    else:
        df2 = df.melt(
            id_vars=[
                "Route",
                "Direction",
            ],
            value_vars=[
                "Average Scheduled Service (Trip Minutes)",
                "Average Stop Distance (Miles)",
                "# Scheduled Trips",
                "Gtfs Availability",
                "Peak Avg Speed",
                "Peak Scheduled Trips",
                "Peak Hourly Freq",
                "Offpeak Avg Speed",
                "Offpeak Scheduled Trips",
                "Trips Per Hour",
            ],
        )
        # Create a decoy column to center all the text
        df2["Zero"] = 0

        df2["combo_col"] = df2.variable.astype(str) + ": " + df2.value.astype(str)
        df2.combo_col = df2.combo_col.str.replace(
            "schedule_and_vp", "Schedule and Realtime Data"
        )
        text_chart = (
            alt.Chart(df2)
            .mark_text()
            .encode(x=alt.X("Zero:Q", axis=None), y=alt.Y("combo_col", axis=None))
        )

        text_chart = text_chart.encode(text="combo_col:N").properties(
            title=f"Route Statistics for Direction {direction}",
            width=500,
            height=300,
        )
        return text_chart

In [ ]:
# create_text_table(table_df, 0)

#### Putting it all together

In [ ]:
def filtered_route_test(
    df: pd.DataFrame,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    # Create dropdown
    routes_list = df["Route"].unique().tolist()

    route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes",
    )
    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=["Route"],
        bind=route_dropdown,
    )

    # Filter for only rows categorized as found in schedule and vp and all_day
    all_day = df.loc[df["Period"] == "All Day"].reset_index(drop=True)

    # Create route stats table for the text tables
    route_stats_df = route_stats(df)

    # Manipulate the df for some of the metrics
    timeliness_df = timeliness_trips(df)

    rt_journey_vp = pct_vp_journey(
        all_day, "% Actual Trip Minutes with 1+ VP per Minute",
    "% Actual Trip Minutes with 2+ VP per Minute",
    )
    sched_journey_vp = pct_vp_journey(
        all_day,"% Scheduled Trip Minutes with 1+ VP per Minute", "% Scheduled Trip Minutes with 2+ VP per Minute"
    )

    avg_scheduled_min_graph = (
        grouped_bar_chart(
            df=all_day,
            color_col="Direction",
            y_col="Average Scheduled Service (trip minutes)",
            offset_col="Direction",
            title=readable_dict["avg_scheduled_min_graph"]["title"],
            subtitle=readable_dict["avg_scheduled_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    timeliness_trips_dir_0 = (
        (
            base_facet_chart(
                timeliness_df.loc[timeliness_df["Direction"] == 0],
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_dir_0_graph"]["title"],
                readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
            )
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    timeliness_trips_dir_1 = (
        (
            base_facet_chart(
                timeliness_df.loc[timeliness_df["Direction"] == 1],
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_dir_1_graph"]["title"],
                readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
            )
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    frequency_graph = (
        frequency_chart(df).add_params(route_selector).transform_filter(route_selector)
    )
    
    speed_graph = (
        base_facet_line(
            df,
            "Speed (MPH)",
            readable_dict["speed_graph"]["title"],
            readable_dict["speed_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    vp_per_min_graph = (
            (
                base_facet_with_ruler_chart(
                    all_day,
                    "Average VP per Minute",
                    "ruler_for_vp_per_min",
                    readable_dict["vp_per_min_graph"]["title"],
                    readable_dict["vp_per_min_graph"]["subtitle"],
                )
            )
            .add_params(route_selector)
            .transform_filter(route_selector)
        )

    rt_vp_per_min_graph = (
        base_facet_circle(
            rt_journey_vp,
            "% of Actual Trip Minutes",
            "Category",
            "ruler_100_pct",
            readable_dict["rt_vp_per_min_graph"]["title"],
            readable_dict["rt_vp_per_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    sched_vp_per_min = (
        base_facet_circle(
            sched_journey_vp,
            "% of Actual Trip Minutes",
            "Category",
            "ruler_100_pct",
            readable_dict["sched_vp_per_min_graph"]["title"],
            readable_dict["rt_vp_per_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    spatial_accuracy = (
        base_facet_with_ruler_chart(
            all_day,
            "% VP within Scheduled Shape",
            "ruler_100_pct",
            readable_dict["spatial_accuracy_graph"]["title"],
            readable_dict["spatial_accuracy_graph"]["title"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    """
    text_dir0 = (
        (create_text_table(route_stats_df, 0))
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    text_dir1 = (
        create_text_table(route_stats_df, 1)
        .add_params(route_selector)
        .transform_filter(route_selector))
  """
    
    chart_list = [
        avg_scheduled_min_graph,
        timeliness_trips_dir_0,
        timeliness_trips_dir_1,
        frequency_graph,
        speed_graph,
        vp_per_min_graph,
        rt_vp_per_min_graph,
        sched_vp_per_min,
        spatial_accuracy,
        
     
    ]

    chart = alt.vconcat(*chart_list).properties(
        resolve=alt.Resolve(
            scale=alt.LegendResolveMap(color=alt.ResolveMode("independent"))
        )
    )
    return chart

In [ ]:
filtered_route_test(df)